In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from data_processing import get_dataset, tokenize_dataset
import torch
import os

In [ ]:
model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train, validation, _ = get_dataset()
train_dataset = tokenize_dataset(train, tokenizer)
validation_dataset = tokenize_dataset(validation, tokenizer)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
args = TrainingArguments(
    output_dir="qwen25_coder_1_5b_instruct",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    save_strategy="steps",
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    greater_is_better=True,
    report_to="wandb",
    fp16=torch.cuda.is_available()
)

os.environ["WANDB_ENTITY"]="VulnRL"

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()
model.save_pretrained("qwen25_coder_1_5b_instruct")
tokenizer.save_pretrained("qwen25_coder_1_5b_instruct")